In [18]:
#Import libraries
import webbrowser
import os
import glob
import chardet
import datetime
import numpy as np
import pandas as pd

In [43]:
stations={
    'Paya Lebar': {
        'code':      '06',
        'latitude':  1.3524, 
        'longitude': 103.9007},
    'Tengah':{
        'code':      '23',
        'latitude':  1.3858,
        'longitude': 103.7114},
    'Changi':{
        'code':      '24',
        'latitude':  1.3678, 
        'longitude': 103.9826},
    'Seletar':{
        'code':      '25',
        'latitude':  1.4166, 
        'longitude': 103.8654},
    'Tai Seng':{
        'code':      '43',
        'latitude':  1.3399, 
        'longitude': 103.8878},
    'Jurong (West)':{
        'code':      '44',
        'latitude':  1.3455 , 
        'longitude': 103.6806 },
    'Ang Mo Kio':{
        'code':      '109',
        'latitude':  1.3764  , 
        'longitude': 103.8492},
    'Clementi':{
        'code':      '50',
        'latitude':  1.3337, 
        'longitude': 103.7768},
    'Admiralty':{
        'code':      '104',
        'latitude':  1.4439, 
        'longitude': 103.7854},
    'Sentosa Island':{
        'code':      '60',
        'latitude':  1.2500, 
        'longitude': 103.8279},
    'Sembawang':{
        'code':      '80',
        'latitude':  1.4252, 
        'longitude': 103.8202},
    'Boon Lay (East)':{
        'code':      '86',
        'latitude':  1.3302, 
        'longitude': 103.7205},
    'Semakau Island':{
        'code':      '102',
        'latitude':  1.1890, 
        'longitude': 103.7680},
    'Pulau Ubin':{
        'code':      '106',
        'latitude':  1.4168, 
        'longitude': 103.9673},
    'East Coast Parkway':{
        'code':      '107',
        'latitude':  1.3134, 
        'longitude': 103.9619},
    'Marina Barrage':{
        'code':      '108',
        'latitude':  1.2799, 
        'longitude': 103.8703},
    'Newton':{
        'code':      '111',
        'latitude':  1.3106, 
        'longitude': 103.8365},
    'Tuas South':{
        'code':      '115',
        'latitude':  1.2938, 
        'longitude': 103.6184},
    'Pasir Panjang':{
        'code':      '116',
        'latitude':  1.2810, 
        'longitude': 103.7540},
    'Jurong Island':{
        'code':      '117',
        'latitude':  1.2542, 
        'longitude': 103.6741},
    'Choa Chu Kang (South)':{
        'code':      '121',
        'latitude':  1.3729, 
        'longitude': 103.722},
    'Khatib':{
        'code':      '122',
        'latitude':  1.4173, 
        'longitude': 103.8249}
}

Missing files:
S121_201505
S111_201611
S111_201610
S111_201609
S106_201402
S106_201401
S102_201411
S102_201401
S86_201812
...
S86_201602
S60_201612
S104_201409
S44_201412
S44_201411
S44_201410
S44_201409
S44_201408
S44_201407

In [50]:
weather_df=pd.DataFrame()
# Append all 1,269 monthly files together
for station in stations.keys():
    df_append = pd.DataFrame()
    code=stations[station]['code']
    path = "../Assets/climate_historical_daily/DAILYDATA_S"+code+"_*.csv"

    for file in glob.glob(path):
        with open(file,'rb') as f:
            enc = chardet.detect(f.read())
        df_temp = pd.read_csv(file,encoding = enc['encoding']) # otherwise will have uft-8 error as is in ISO-8859-1 format
        df_temp = df_temp.rename(columns=str.lower) # columns names in different case between files
        df_append = pd.concat(objs =[df_append,df_temp],axis=0)
        
    df_append['station_check']=station
    df_append['station_code']=code
    df_append['latitude']=stations[station]['latitude']
    df_append['longitude']=stations[station]['longitude']
    weather_df = pd.concat(objs =[weather_df,df_append],axis=0)

In [51]:
weather_df['check']=weather_df['station']==weather_df['station_check']
weather_df['check'].value_counts()

True    38620
Name: check, dtype: int64

In [52]:
weather_df=weather_df.drop(['station_check','check'],axis=1)
weather_df

,station,year,month,day,daily rainfall total (mm),highest 30 min rainfall (mm),highest 60 min rainfall (mm),highest 120 min rainfall (mm),mean temperature (°c),maximum temperature (°c),minimum temperature (°c),mean wind speed (km/h),max wind speed (km/h),station_code,latitude,longitude
0,Paya Lebar,2018,1,1,33.2,—,—,—,25.2,27.3,23.8,7.2,27.7,06,1.3524,103.9007
1,Paya Lebar,2018,1,2,10.2,—,—,—,25.8,28.4,24.1,8.6,25.9,06,1.3524,103.9007
2,Paya Lebar,2018,1,3,62.2,—,—,—,26.8,32.1,24.5,7.6,31.3,06,1.3524,103.9007
3,Paya Lebar,2018,1,4,0.3,—,—,—,26.9,29.5,25.0,9.4,33.5,06,1.3524,103.9007
4,Paya Lebar,2018,1,5,1.5,—,—,—,26.8,30.0,24.3,9.0,24.1,06,1.3524,103.9007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Khatib,2015,9,26,0.0,0.0,0.0,0.0,28.7,32.5,24.6,5.1,26.3,122,1.4173,103.8249
26,Khatib,2015,9,27,0.0,0.0,0.0,0.0,28.6,32.6,25.2,3.9,23.4,122,1.4173,103.8249
27,Khatib,2015,9,28,23.0,16.2,19.6,21.2,27.1,31.9,23.5,3.4,30.6,122,1.4173,103.8249
28,Khatib,2015,9,29,0.0,0.0,0.0,0.0,27.8,31.7,24.6,3.6,19.1,122,1.4173,103.8249


In [53]:
new_columns_dict = {
    'station': 'station',
    'year':'year',
    'month': 'month',
    'day': 'day',
    'daily rainfall total (mm)': 'total_daily_rainfall',
    'highest 30 min rainfall (mm)': 'highest_30min_rainfall',
    'highest 60 min rainfall (mm)': 'highest_60min_rainfall',
    'highest 120 min rainfall (mm)': 'highest_120min_rainfall',
    'mean temperature (°c)': 'mean_temp',
    'maximum temperature (°c)': 'max_temp',
    'minimum temperature (°c)': 'min_temp',
    'mean wind speed (km/h)': 'mean_wind_sp',
    'max wind speed (km/h)': 'max_wind_sp'
}

weather_df.rename(columns=new_columns_dict, inplace=True)

In [54]:
weather_df.insert(4,'date',
                  pd.to_datetime(dict(year=weather_df['year'],
                                      month=weather_df['month'],
                                      day=weather_df['day'])))

In [55]:
col_list=['total_daily_rainfall','highest_30min_rainfall','highest_60min_rainfall','highest_120min_rainfall',
          'mean_temp','max_temp','min_temp',
          'mean_wind_sp','max_wind_sp']
for col in col_list:
    weather_df[col] = weather_df[col].map(lambda x: str(x).strip()).replace(['—','-'], np.nan).map(lambda x: float(x)) #different '-'s

In [56]:
weather_df['rainy_day'] = weather_df['total_daily_rainfall'].apply(lambda x: 1 if x>=0.2 else 0)

In [57]:
weather_df

,station,year,month,day,date,total_daily_rainfall,highest_30min_rainfall,highest_60min_rainfall,highest_120min_rainfall,mean_temp,max_temp,min_temp,mean_wind_sp,max_wind_sp,station_code,latitude,longitude,rainy_day
0,Paya Lebar,2018,1,1,2018-01-01,33.2,NaN,NaN,NaN,25.2,27.3,23.8,7.2,27.7,06,1.3524,103.9007,1
1,Paya Lebar,2018,1,2,2018-01-02,10.2,NaN,NaN,NaN,25.8,28.4,24.1,8.6,25.9,06,1.3524,103.9007,1
2,Paya Lebar,2018,1,3,2018-01-03,62.2,NaN,NaN,NaN,26.8,32.1,24.5,7.6,31.3,06,1.3524,103.9007,1
3,Paya Lebar,2018,1,4,2018-01-04,0.3,NaN,NaN,NaN,26.9,29.5,25.0,9.4,33.5,06,1.3524,103.9007,1
4,Paya Lebar,2018,1,5,2018-01-05,1.5,NaN,NaN,NaN,26.8,30.0,24.3,9.0,24.1,06,1.3524,103.9007,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Khatib,2015,9,26,2015-09-26,0.0,0.0,0.0,0.0,28.7,32.5,24.6,5.1,26.3,122,1.4173,103.8249,0
26,Khatib,2015,9,27,2015-09-27,0.0,0.0,0.0,0.0,28.6,32.6,25.2,3.9,23.4,122,1.4173,103.8249,0
27,Khatib,2015,9,28,2015-09-28,23.0,16.2,19.6,21.2,27.1,31.9,23.5,3.4,30.6,122,1.4173,103.8249,1
28,Khatib,2015,9,29,2015-09-29,0.0,0.0,0.0,0.0,27.8,31.7,24.6,3.6,19.1,122,1.4173,103.8249,0


In [58]:
weather_df.to_csv('../assets/climate_historical_daily_2014-2018.csv',index=False)